In [1]:
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [36]:
def dsigmoid(x):
    s=sigmoid(x)
    return s*(1-s)

Network parameters is represnted as [$\alpha^0,\theta^0_{0},\theta^0_{1},\alpha^1,\theta^1_{0},\theta^1_{1},\alpha^2,\theta^2_{0},\theta^2_{1},\alpha^3,\theta^3_{0},\theta^3_{1}$]

In [4]:
def network_function(network_params,x):
    first_term = network_params[0] * sigmoid(network_params[1]+ network_params[2] * x)
    second_term = network_params[3] * sigmoid(network_params[4] + network_params[5]* x)
    third_term = network_params[6] * sigmoid(network_params[7] + network_params[8]* x)
    fourth_term = network_params[9] * sigmoid(network_params[10] + network_params[11]* x)
    return first_term + second_term + third_term + fourth_term

In [5]:
def inner_product(f,g,network_params):
    points = np.linspace(0,2 * np.pi,100)
    sum = 0
    for point in points:
        sum = sum + f(network_params,point) * g(network_params,point)
    return sum

In [12]:
#Cost function
def norm_of_u_minus_sin_squared(network_params):
    u = network_function
    #x is input to sine and network
    u_minus_sin = lambda network_params,x: u(network_params,x) - np.sin(x)
    return  (inner_product(u_minus_sin,u_minus_sin,network_params)) / 200.0

In [25]:
data = np.linspace(0,2 * np.pi,100)

In [31]:
def gradcost(theta):
    gradvect=np.zeros(12)
    for x in data:
        dcostdmu=(network_function(theta,x)-np.sin(x))
        for i in range(0,12,3):
            term=dcostdmu*theta[i]*dsigmoid(theta[i+1]+theta[i+2]*x)
            gradvect[i]+=dcostdmu*sigmoid(theta[i+1]+theta[i+2]*x)
            gradvect[i+1]+=term
            gradvect[i+2]+=term*x
    return 1.0/100*gradvect

In [32]:
#Does Forward difference in each dimension
def eval_numerical_gradient(f, x):
    """ 
    a naive implementation of numerical gradient of f at x
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """
    fx = f(x) # evaluate function value at original point
    grad = np.zeros(x.shape)
    h = 0.00001

    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:

        # evaluate function at x+h
        ix = it.multi_index
        old_value = x[ix]
        x[ix] = old_value + h # increment by h
        fxh = f(x) # evalute f(x + h)
        x[ix] = old_value # restore to previous value (very important!)
        # compute the partial derivative
        grad[ix] = (fxh - fx) / h # the slope
        it.iternext() # step to next dimension
    return grad

In [33]:
mytheta0 = np.random.rand(12)

In [34]:
numgrad = eval_numerical_gradient(norm_of_u_minus_sin_squared,mytheta0)

In [37]:
print (numgrad-gradcost(mytheta0))

[  3.83123405e-06  -3.96846139e-07  -5.69111473e-06   4.31887614e-06
  -7.26066509e-08  -4.96932604e-07   1.66334332e-06  -1.16705629e-07
  -2.94403583e-06   3.22880348e-06  -3.73808780e-07  -6.62361799e-06]


In [54]:
a_old = np.random.rand(12) # start with random coefficients instead of zeros
print ("Starting with initial coefficients", a_old)
h = 10**(-7) # step size
tol_1 = 10**(-10) # stop if change in coefficients is less than tol_1
tol_2 = 10**(-10) # stop if change in total cost is less than tol_2
max_steps = 500 # stop if total number of steps exceeds max_steps

a_list = [a_old]
z_list = [norm_of_u_minus_sin_squared(a_old)]
unit_grad = np.linalg.norm(eval_numerical_gradient(norm_of_u_minus_sin_squared,a_old))
a_new = a_old - h * unit_grad
a_list.append(a_new)
z_list.append(norm_of_u_minus_sin_squared(a_new))

for i in range(max_steps):
    if sum((a_new-a_old)**2) < tol_1:
        print ("domain values within tolerance ", tol_1)
        break
    if abs(z_list[-1] - z_list[-2])<tol_2:
        print ("cost values within tolerance ", tol_2)
        break
    a_old = a_new
    unit_grad = np.linalg.norm(eval_numerical_gradient(norm_of_u_minus_sin_squared,a_old))
    a_new = a_old - h * unit_grad
    a_list.append(a_new)
    z_list.append(norm_of_u_minus_sin_squared(a_new))


print ("Local minimum occurs at:", a_new)
print ("Number of steps:", len(a_list))
print ("minimum value is:", norm_of_u_minus_sin_squared(a_new))

Starting with initial coefficients [ 0.14972113  0.69251867  0.5541395   0.09615764  0.18080972  0.71975808
  0.84516115  0.37957641  0.61062095  0.08438766  0.3494913   0.72804208]
domain values within tolerance  1e-10
Local minimum occurs at: [ 0.14972094  0.69251847  0.5541393   0.09615745  0.18080953  0.71975789
  0.84516096  0.37957621  0.61062075  0.08438746  0.34949111  0.72804189]
Number of steps: 2
minimum value is: 0.843982972778


In [55]:
x_0 = np.linspace(0, 2 * np.pi,100)
y_0 = []
y_1 = []
for p in x_0:
    y_0.append(network_function(a_new,p))
    y_1.append(np.sin(p))

In [56]:
fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.9, 0.7])
axes.plot(x_0,y_0,label ="Gradient Descent",c = "red");
axes.plot(x_0,y_1, label = "Sine Function", c = "blue")
axes.legend(loc = 3)

Perhaps should use gradient descent that uses parabola